In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

def find_investment_windows(df, date_column='date', balance_column='balance', min_days=2):
    """
    Find optimal investment windows based on account balance data.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing date and balance columns
    date_column : str
        Name of the date column
    balance_column : str
        Name of the balance column
    min_days : int
        Minimum number of days for an investment window

    Returns:
    --------
    list of dict
        Each dict contains details about an investment window
    """
    # Ensure the dataframe is sorted by date
    df = df.sort_values(by=date_column).reset_index(drop=True)

    # Initialize variables
    investment_windows = []

    # Find all possible minimum balances for each possible starting day
    for start_idx in range(len(df) - min_days + 1):
        for end_idx in range(start_idx + min_days - 1, len(df)):
            window_df = df.iloc[start_idx:end_idx + 1]
            min_balance = window_df[balance_column].min()

            # Only consider windows where we have a positive balance to invest
            if min_balance > 0:
                days = (df.iloc[end_idx][date_column] - df.iloc[start_idx][date_column]).days + 1

                # Calculate a score that values both amount and duration
                # This is a simple score function that multiplies amount by days
                score = min_balance * days

                window_info = {
                    'start_date': df.iloc[start_idx][date_column],
                    'end_date': df.iloc[end_idx][date_column],
                    'amount': min_balance,
                    'days': days,
                    'score': score
                }

                investment_windows.append(window_info)

    # Sort investment windows by score in descending order
    investment_windows.sort(key=lambda x: x['score'], reverse=True)

    return investment_windows

def optimize_non_overlapping_windows(windows, top_n=10):
    """
    Find the optimal set of non-overlapping investment windows

    Parameters:
    -----------
    windows : list of dict
        List of investment windows from find_investment_windows
    top_n : int
        Number of top windows to consider

    Returns:
    --------
    list of dict
        Optimal set of non-overlapping investment windows
    """
    # Start with top N windows by score
    candidate_windows = windows[:min(top_n, len(windows))]

    # Sort by start date
    candidate_windows.sort(key=lambda x: x['start_date'])

    # Dynamic programming approach to find optimal non-overlapping windows
    n = len(candidate_windows)
    if n == 0:
        return []

    # dp[i] represents the maximum score we can get by considering windows 0...i
    dp = [0] * n
    dp[0] = candidate_windows[0]['score']
    prev = [-1] * n  # To track which previous window was chosen

    for i in range(1, n):
        # Option 1: Don't include current window
        option1 = dp[i-1]

        # Option 2: Include current window
        option2 = candidate_windows[i]['score']
        last_non_overlapping = -1

        # Find the last non-overlapping window
        for j in range(i-1, -1, -1):
            if candidate_windows[j]['end_date'] < candidate_windows[i]['start_date']:
                option2 += dp[j]
                last_non_overlapping = j
                break

        # Choose the better option
        if option1 >= option2:
            dp[i] = option1
            prev[i] = prev[i-1]
        else:
            dp[i] = option2
            prev[i] = last_non_overlapping

    # Reconstruct the solution
    optimal_windows = []
    i = n - 1

    while i >= 0:
        if prev[i] == prev[i-1] and i > 0:
            i -= 1
        else:
            optimal_windows.append(candidate_windows[i])
            i = prev[i]

    # Reverse to get chronological order
    optimal_windows.reverse()

    return optimal_windows

def visualize_investment_windows(df, windows, date_column='date', balance_column='balance'):
    """
    Visualize the investment windows on top of the account balance

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing date and balance columns
    windows : list of dict
        List of investment windows
    date_column : str
        Name of the date column
    balance_column : str
        Name of the balance column
    """
    plt.figure(figsize=(12, 6))

    # Plot the account balance
    plt.plot(df[date_column], df[balance_column], label='Max. Asset Allocation',)

    # Plot the investment windows
    for i, window in enumerate(windows):
        plt.axvspan(window['start_date'], window['end_date'],
                   alpha=0.2, color='#361B07', #f'C{i+1}',
                   label=f"Window {i+1}: ${window['amount']:.2f} for {window['days']} days")

        # Plot the investment amount as a horizontal line
        plt.hlines(window['amount'], window['start_date'], window['end_date'],
                  colors='#f2a900', linestyles='dashed') #f'C{i+1}'

    plt.xlabel('Date')
    plt.ylabel('Available ($ in B)')
    plt.title('Available capital & Max. ROI investment window')
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    return plt

# Example usage
if __name__ == "__main__": # run only when called script directly

    # Load the data
    running_balances = pd.read_pickle('running_balances.pkl')
    df = running_balances[
        (running_balances['TransactionClass'] == 'Commercial Paper') &
        (running_balances['TransactionDate'] <= '2025-06-30') &
        (running_balances['TransactionDate'] > '2025-01-21')
    ][['TransactionDate', 'Available']].rename(
        columns={'TransactionDate': 'date', 'Available': 'balance'}
    )

    # Find investment windows
    windows = find_investment_windows(df, min_days=5)  # Minimum 30 days for investment
    print(f"Found {len(windows)} potential investment windows")

    # Get top 5 windows by score
    top_windows = windows[:5]
    print("\nTop 5 Investment Windows:")
    for i, window in enumerate(top_windows):
        print(f"{i+1}. {window['start_date'].date()} to {window['end_date'].date()}: "
              f"${window['amount']:.2f} for {window['days']} days (Score: {window['score']:.2f})")

    # Find optimal non-overlapping windows
    optimal_windows = optimize_non_overlapping_windows(windows, top_n=20)
    print("\nOptimal Non-overlapping Investment Windows:")
    total_value = 0
    for i, window in enumerate(optimal_windows):
        value = window['amount'] * window['days']
        total_value += value
        print(f"{i+1}. {window['start_date'].date()} to {window['end_date'].date()}: "
              f"${window['amount']:.2f} for {window['days']} days (Value: ${value:.2f})")
    print(f"\nTotal value of optimal investment strategy: ${total_value:.2f}")

    # Visualize the investment windows
    plt_figure = visualize_investment_windows(df, optimal_windows)
    plt.show()

In [ ]:
# Sort windows by start_date
windows.sort(key=lambda x: x['start_date'])

total_value = 0
for i, window in enumerate(windows):
      value = window['amount'] * window['days']
      total_value += value
      print(f"{i+1}. {window['start_date'].date()} to {window['end_date'].date()}: "
            f"${(window['amount']/1000000):,.2f}M for {window['days']} days (Value: ${value/1000000:,.2f}M)")